<a href="https://colab.research.google.com/github/jinyoungkim0214/data-augmentation/blob/main/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
transforms.RandomResizedCrop(224),           # 무작위 크롭 후 224x224
transforms.RandomHorizontalFlip(p=0.5),      # 50% 확률로 수평 뒤집기
transforms.RandomVerticalFlip(p=0.5),        # 50% 확률로 수직 뒤집기
transforms.RandomRotation(degrees=30),       # 최대 30도 무작위 회전
transforms.ColorJitter(brightness=0.2,       # 밝기, 대비, 채도, 색조 조정
                                   contrast=0.2,
                                   saturation=0.2,
                                   hue=0.1),
transforms.RandomAffine(degrees=0,            # 이동 및 스케일링
                                   translate=(0.1, 0.1),
                                   scale=(0.8, 1.2)),
transforms.RandomErasing(p=0.3,              # 30% 확률로 무작위 영역 지우기
                                    scale=(0.02, 0.2),
                                    ratio=(0.3, 3.3)),

materials classification 성능 향상시킬 만한 방법들

materials classification 파일에는 random resized crop만 됨 -> augmentation 코드 하나씩 넣어보면서 성능 비교해볼 수 있음


activation function 바꾸거나, 다른 network embedding 시키거나 하는 방법도 적용 가능

SGD를 아담이나 아담W로 바꿔보기 가능

아담:
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)

아담W:
optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=1e-4)


Cross entropy loss 말고 다른 loss 추가(재료적 특성 반영할 만한 새로운 loss 설계)
class 나눠놓고 아예 잘못 대분류하면 loss를 크게 준다거나.

In [ ]:
# 스타일 손실 계산 함수 #새로운 loss를 추가하는 방식이긴 한데, 훨씬 정교하게 짤 필요는 있어 보임.
def gram_matrix(feature_map):
    """특징 맵의 Gram 행렬을 계산"""
    b, c, h, w = feature_map.size()  # 배치, 채널, 높이, 너비
    features = feature_map.view(b, c, h * w)  # [b, c, h*w]
    gram = torch.bmm(features, features.transpose(1, 2))  # [b, c, c]
    return gram / (c * h * w)  # 정규화

class StyleLoss(nn.Module):
    def __init__(self):
        super(StyleLoss, self).__init__()
        self.mse_loss = nn.MSELoss()

    def forward(self, input_features, target_features):
        """입력과 타겟 특징 맵의 스타일 손실 계산"""
        input_gram = gram_matrix(input_features)
        target_gram = gram_matrix(target_features)
        return self.mse_loss(input_gram, target_gram)

# 모델 수정: 중간 특징 맵 추출
class BaselineModel1(nn.Module):
    def __init__(self):
        super().__init__()
        self.fe = ResNetFeatureExtractor(resnet_type="resnet101", in_channels=3)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, 47)

        # 스타일 손실을 위한 중간 레이어 저장
        self.intermediate_features = []
        self.fe.feature_extractor.layer3.register_forward_hook(self._hook_fn)

    def _hook_fn(self, module, input, output):
        self.intermediate_features.append(output)

    def forward(self, x):
        self.intermediate_features = []  # 초기화
        x = self.fe(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x, self.intermediate_features

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()
style_loss_fn = StyleLoss()

# 옵티마이저 (AdamW, 이전 설정 유지)
optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# 손실 기록용 리스트
training_loss_history = []
test_loss_history = []

# 훈련 루프
net = BaselineModel1()
style_weight = 1e4  # 스타일 손실 가중치 (조정 가능)
for epoch in range(2):  # 2 epoch
    # 학습 단계
    running_loss = 0.0
    running_style_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        # 모델 출력 (분류 출력 + 중간 특징 맵)
        outputs, intermediate_features = net(inputs)

        # 분류 손실
        cls_loss = criterion(outputs, labels)

        # 스타일 손실 (layer3의 특징 맵 사용)
        if intermediate_features:
            style_loss = style_loss_fn(intermediate_features[0], intermediate_features[0])  # 입력 자체와 비교
            total_loss = cls_loss + style_weight * style_loss
        else:
            total_loss = cls_loss

        total_loss.backward()
        optimizer.step()

        running_loss += cls_loss.item()
        running_style_loss += style_loss.item() if intermediate_features else 0.0

        if i % 10 == 9:
            print(f'[{epoch + 1}, {i + 1:5d}] cls_loss: {running_loss / 200:.3f}, style_loss: {running_style_loss / 200:.3f}')
            training_loss_history.append(running_loss / 200)
            running_loss = 0.0
            running_style_loss = 0.0

    # 테스트 단계
    running_test_loss = 0.0
    net.eval()
    with torch.no_grad():
        for i, test_data in enumerate(testloader, 0):
            test_images, test_labels = test_data
            test_outputs, _ = net(test_images)
            test_loss = criterion(test_outputs, test_labels)
            running_test_loss += test_loss.item()

    avg_test_loss = running_test_loss / (i + 1)
    test_loss_history.append(avg_test_loss)

    # 학습률 스케줄러 업데이트
    scheduler.step(avg_test_loss)

    print(f'Epoch {epoch + 1} - Test Loss: {avg_test_loss:.3f}')

print('학습 끝!')
```